
# ※ 영단어 database를 이용하여 마지막 글자를 예측하는 프로그램 만들기  # 

## LSTM 학습 시작




In [21]:
import tensorflow as tf
import numpy as np

**사용할 library import**

In [22]:
char_arr = ['a', 'b', 'c', 'd', 'e', 'f', 'g',
            'h', 'i', 'j', 'k', 'l', 'm', 'n',
            'o', 'p', 'q', 'r', 's', 't', 'u',
            'v', 'w', 'x', 'y', 'z']

num_dic = {n: i for i, n in enumerate(char_arr)}
dic_len = len(num_dic)

**글자 를 사용하므로 각 글자를 숫자에 대응하게 mapping.**
      ex) a - 1
          b - 2
          c - 3
          .
          .
          .
      - enumerate : 리스트가 있는 경우 순서와 해당 리스트 값을 전달
      - len : 입력값의 길이(요소 전체 갯수)를 반환함

In [23]:
seq_data = ['helps', 'warm', 'deep', 'dive', 'cold', 'cool', 'load',
           'love', 'kiss', 'kind', 'work', 'hide']

test_data = ['helps', 'work', 'dump']

**사용할 Data set**

* training 단어 목록 = seq_data
* test 단어 목록 = test_data

In [24]:
def make_batch(word):
    input_batch = []
    target_batch = []

    input = [num_dic[n] for n in word[:-1]]
    target = num_dic[word[-1]]  
    input_batch = np.eye(dic_len)[input]
    dim0, dim1 = np.shape(input_batch)
    input_batch = np.reshape(input_batch, (-1, dim0, dim1))
    target_batch.append(target)

    return input_batch, target_batch

**입력 받은 단어 알파벳 -> 숫자 나열 input과 target으로 바꾸기**

        ex) input : deep - [3, 4, 4] (target이 될 p 제외)
            target : deep - p의 10

***
**mapped data encoding**

  input_batch : 한 단어당 한 2차원 vector가 대응되고 가로 세로 길이는 alphabet 수와 같음
       ex) input = deep[3, 4, 4, 10]
             -   [[0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0]
                  [0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0]
                  [0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0]
                  [0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0]]
                  
  target_batch : target 숫자를 그대로 list에 붙이며 저장
         - tf.append : input을 list 뒤에 붙임
         - np.eye(dic_len)[input] : 각 줄의 input 자리가 1인 dic_len크기의 2차
                                    원배열 생성 

In [25]:
learning_rate = 0.01
n_hidden = 128
total_epoch = 30
    # epoch은 트레이닝 반복 횟수
n_input = n_class = dic_len

**옵션 설정** 

* training 에 필요한 hyperparameter 결정
       - node 수 : 128
       - 최적화 위한 반복 수 : 30

In [26]:
X = tf.placeholder(tf.float32, [None, None, n_input])
Y = tf.placeholder(tf.int32, [None])
W = tf.Variable(tf.random_normal([n_hidden, n_class]))
    # W는 hidden layer 수만큼
b = tf.Variable(tf.random_normal([n_class]))
    # W, b는 바뀌는 값이라서 Variable로 선언

**placeholder와 variable 생성** 

In [27]:
cell1 = tf.nn.rnn_cell.BasicLSTMCell(n_hidden)
cell1 = tf.nn.rnn_cell.DropoutWrapper(cell1, output_keep_prob=0.5)
    # overfit 방지하기 위해 dropout
cell2 = tf.nn.rnn_cell.BasicLSTMCell(n_hidden)
multi_cell = tf.nn.rnn_cell.MultiRNNCell([cell1, cell2])
    # 성능 향상을 위해 multi cell 사용
outputs, states = tf.nn.dynamic_rnn(multi_cell, X, dtype=tf.float32)

ValueError: Variable rnn/multi_rnn_cell/cell_0/basic_lstm_cell/kernel already exists, disallowed. Did you mean to set reuse=True in VarScope? Originally defined at:

  File "<ipython-input-8-ceb723be90d6>", line 7, in <module>
    outputs, states = tf.nn.dynamic_rnn(multi_cell, X, dtype=tf.float32)
  File "/home/dabuntu/Apps/anaconda/lib/python3.6/site-packages/IPython/core/interactiveshell.py", line 2862, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "/home/dabuntu/Apps/anaconda/lib/python3.6/site-packages/IPython/core/interactiveshell.py", line 2802, in run_ast_nodes
    if self.run_code(code, result):


**RNN을 위한 Cell 생성**
    - LSTM cell 2개를 만들어 조합하여 사용함. 
    - overfitting 방지를 위해 dropout을 병렬로 행함
    
***    
    
**cell을 이용하여 순환신경망 만들기**
    - dynamic_rnn 함수 사용 : (cell, input, data type)
                              output = output tensor와 마지막 state를 return

In [ ]:
outputs = tf.transpose(outputs, [1, 0, 2])
outputs = outputs[-1]
model = tf.matmul(outputs, W) + b

**최종 결과를 one-hot encoding 형식으로 만듬**
        - transpose : 행렬을 전치. [[0, 1],   ->    [[0, 2, 4],
                                   [2, 3],          [1, 3, 5] 
                                   [4, 5]]

In [ ]:
cost = tf.reduce_mean(tf.nn.sparse_softmax_cross_entropy_with_logits(
                logits=model, labels=Y))
    # softmax가 아니라 다른 함수 사용 -> 성능 위해서
optimizer = tf.train.AdamOptimizer(learning_rate).minimize(cost)

**cost 정의** 
    - reduce_mean 사용, sparse_softmax_cross_entropy_with_logits 사용

**optimizer**
    - AdamOptimizer 사용 back propagation

In [ ]:
sess = tf.Session()
sess.run(tf.global_variables_initializer())

for epoch in range(total_epoch):
    for word in range(len(seq_data)):
        input_batch, target_batch = make_batch(seq_data[word])
        _, loss = sess.run([optimizer, cost],
                       feed_dict={X: input_batch, Y: target_batch})

    print('Epoch:', '%04d' % (epoch + 1),
          'cost =', '{:.6f}'.format(loss))

print('최적화 완료!')

**학습을 위한 최적화 진행**

* 신경망 모델 학습 (session run) -  variable initialize 필수
* epoch=30
       - epoch 가 지남에 따라 cost의 변화, 발전을 print하여 확인
         최적화 완료 후 완료를 알리는 문구 print
       
* batch - 단어 하나가 한 batch 이므로 반복문 사용하여 매번 갱신

## 결과 확인


In [ ]:
prediction = tf.cast(tf.argmax(model, 1), tf.int32)
prediction_check = tf.equal(prediction, Y)

**target label 값이 정수이므로 예측값도 정수로 변경**

  정수끼리 그대로 비교 후 각 단어 별 비교값을 평균내어 accuracy 측정
  측정을 위한 prediction, accuracy 묘사

In [ ]:
acc = 0
predict = []
for i in range(len(test_data)):
    input_batch, target_batch = make_batch(test_data[i])
    pred, accuracy_val = sess.run([prediction, prediction_check],
                               feed_dict={X: input_batch, Y: target_batch})
    acc = acc + accuracy_val
    predict.append(pred)
acc = acc / len(test_data)
acc = acc * 100

**test data로 batch 생성 후 predict와 accuracy 값 얻어냄**

* accuracy : 각 단어마다 prediction과 target을 비교한 값을 평균내어 측정

In [ ]:
predict_words = []
for idx, val in enumerate(test_data):
    last_char = char_arr[predict[idx][0]]
    predict_words.append(val[:-1] + last_char)
    
print('\n=== 예측 결과 ===')
print('입력값:', [w[:-1] + ' ' for w in test_data])
print('예측값:', predict_words)
print('정확도: %.2f' % acc + '%')

**predict word를 출력하여 입력과 비교, 결과를 시각적으로 확인하고 accuracy 출력**

     seq_data 내에 prediction을 진행할 단어를 가져와 그 단어에 해당하는 prediction alpabet을 last_char로 저장
     
     단어의 앞에 세 글자와 last_char을 합쳐 predict_words로 출력
     
       - word[:3] : word 내에 index 3 앞에서 잘라 그 앞 부분만 의미.